In [19]:
#!pip uninstall visualkeras

In [20]:
#!pip install scikit-learn

In [21]:
#!pip install tensorflow-gpu

In [22]:
#!pip install pandas

In [23]:
#!pip install wandb

In [24]:
#!pip install numpy

In [25]:
#!pip install openpyxl

In [26]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox


#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

#Librerias estandar (Extras)
import re
import os
import time
import random
from datetime import datetime

In [27]:
"""
DIRECTORIOS DEL PROYECTO
"""
path_base = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis'   # Directorio del proyecto
path_imagenes = 'F:/GOES/'      

In [28]:
!python --version
print(tf. __version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#Limitamos el GPU, en caso se necesite
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

Python 3.9.11
2.8.0
Num GPUs Available:  1


In [29]:
#conda list cudnn

In [30]:
#conda list cudatoolkit

In [31]:
"""
Metodos para realizar el entrenamient - evaluacion del modelo
"""

'\nMetodos para realizar el entrenamient - evaluacion del modelo\n'

In [32]:
def crearModelo2D(p,run):    
    # Imagen
    input_1 = tf.keras.layers.Input(shape=(p['margen'][run],p['margen'][run],p['canales'][run]))
    
    # Convulutional layers
    rescaling = tf.keras.layers.Rescaling(1./65536)(input_1)
    conv2d_1 = tf.keras.layers.Conv2D(128, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(rescaling)
    mxPool_1 = tf.keras.layers.MaxPooling2D()(conv2d_1)
    dropout_1  = tf.keras.layers.Dropout(0.2)(mxPool_1)
    
    conv2d_2 = tf.keras.layers.Conv2D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    mxPool_2 = tf.keras.layers.MaxPooling2D()(conv2d_2)
    dropout_2  = tf.keras.layers.Dropout(0.1)(mxPool_2)
    
    #conv2d_3 = tf.keras.layers.Conv2D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    #mxPool_3 = tf.keras.layers.MaxPooling2D()(conv2d_3)
    #dropout_3  = tf.keras.layers.Dropout(0.2)(mxPool_3)
    
    #conv2d_4 = tf.keras.layers.Conv2D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_3)
    #mxPool_4 = tf.keras.layers.MaxPooling2D()(conv2d_4)
    #dropout_4  = tf.keras.layers.Dropout(0.2)(mxPool_4)
    
    conv2d_5 = tf.keras.layers.Conv2D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_2)
    
    
    # Flatten layer :
    flatten = tf.keras.layers.Flatten()(conv2d_5)
    
    final = flatten
    listConcat = [flatten]
    listInputs = [input_1]
    
    if len(p['inputs'])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat)
        
    dense_1 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(final)
    #dense_2 = tf.keras.layers.Dense(units=16, activation=tf.keras.activations.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_1)
    
        
    # output
    if p['redTipo'] == 'regresion':
        output = tf.keras.layers.Dense(units=1)(dense_3)
        dimOutput = 1
    elif p['redTipo'] == 'clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_3)#units=1, activation=tf.keras.activations.relu)(dense_3)
        dimOutput = 2
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    

    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])
        
    #print(full_model.summary())
    return full_model

In [33]:
def crearModelo3D(p,run):        
    # Imagen
    input_1 = tf.keras.layers.Input(shape=(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
    
    # Convulutional layers
    rescaling = tf.keras.layers.Rescaling(1./65536)(input_1)
    conv3d_1 = tf.keras.layers.Conv3D(128, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(rescaling)
    #mxPool_1 = tf.keras.layers.MaxPooling3D()(conv3d_1)
    dropout_1  = tf.keras.layers.Dropout(0.2)(conv3d_1)
    
    conv3d_2 = tf.keras.layers.Conv3D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    mxPool_2 = tf.keras.layers.MaxPooling3D()(conv3d_2)
    dropout_2  = tf.keras.layers.Dropout(0.1)(mxPool_2)
    
    #conv2d_3 = tf.keras.layers.Conv3D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    #mxPool_3 = tf.keras.layers.MaxPooling3D()(conv2d_3)
    #dropout_3  = tf.keras.layers.Dropout(0.2)(mxPool_3)
    
    #conv2d_4 = tf.keras.layers.Conv3D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_3)
    #mxPool_4 = tf.keras.layers.MaxPooling3D()(conv2d_4)
    #dropout_4  = tf.keras.layers.Dropout(0.2)(mxPool_4)
    
    conv3d_5 = tf.keras.layers.Conv3D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_2)
    
    
    # Flatten layer :
    flatten = tf.keras.layers.Flatten()(conv3d_5)
    
    final = flatten
    listConcat = [flatten]
    listInputs = [input_1]
    
    if len(p['inputs'])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat)
        
    dense_1 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(final)
    #dense_2 = tf.keras.layers.Dense(units=16, activation=tf.keras.activations.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_1)
    
        
    # output
    if p['redTipo'] == 'regresion':
        output = tf.keras.layers.Dense(units=1)(dense_3)
        dimOutput = 1
    elif p['redTipo'] == 'clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_3)#units=1, activation=tf.keras.activations.relu)(dense_3)
        dimOutput = 2
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    

    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])
        
    #print(full_model.summary())
    return full_model

In [34]:
def getMetrics(modelType, lr, paciencia):
    
    if modelType == 'clasificacion':    
        optimizer = keras.optimizers.Adam(learning_rate=lr) 
        
        #BinaryCrossentropy() #CategoricalCrossentropy()      
        loss_fn= keras.losses.BinaryCrossentropy()
        train_acc_metric = keras.metrics.BinaryCrossentropy()
        val_acc_metric = keras.metrics.BinaryCrossentropy()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", patience=paciencia, mode="max")  
 
        
        metrics = ['acc', keras.metrics.TruePositives(),
                         keras.metrics.TrueNegatives(),
                         keras.metrics.FalsePositives(),
                         keras.metrics.FalseNegatives()]
        

    elif modelType == 'regresion':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        loss_fn=keras.losses.MeanSquaredError()
        train_acc_metric = keras.metrics.MeanSquaredError()
        val_acc_metric = keras.metrics.MeanSquaredError()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_mse", patience=paciencia, mode="max")                                            
        metrics = ['mse']
        
        
    else:
        print('No se pudo crear las metricas')
        return -1    
         
        
    logs = Callback()
    callbacks = [logs]                     
    if paciencia:
        callbacks.append(early_stopping)
        
    metrics = {'optimizer': optimizer, 'loss_fn':loss_fn,'train_acc_metric': train_acc_metric,
               'val_acc_metric': val_acc_metric, 'metrics': metrics,'callbacks': callbacks}
    
    return metrics
        

In [35]:
def crearModelo(params,run):
    
    if params['meanMatrizImagen']:
        print(f"Creando modelo 2D")
        print(f"HP :(T:{params['tiempos'][run]} - M:{params['margen'][run]} - C:{params['canales'][run]}) y  tipo ({params['redTipo']})")
        modelo = crearModelo2D(params,run)
    else:
        print(f"Creando modelo 3D")
        print(f"HP :(T:{params['tiempos'][run]} - M:{params['margen'][run]} - C:{params['canales'][run]}) y  tipo ({params['redTipo']})")
        modelo = crearModelo3D(params,run)

    return modelo

In [86]:
def applyDA(img, DA):
    if DA == 1:
        return tf.image.flip_left_right(img)
    elif DA == 2:
        return tf.image.flip_up_down(img)
    elif DA == 3:
        img = tf.image.flip_left_right(img)
        return tf.image.flip_up_down(img)   
    else:
        return img

In [87]:
#Transformamos un filename tensor en una imagen
def read_png_file(item, value, p,run, path_base, products, times, DA=0):
    # imagenData[0] = XO     # imagenData[1] = XA     # imagenData[2] = Fecha
    imagenData = tf.strings.split(item['imagen'], sep='--')
    size = int(p['margen'][run] / 2)

    timeJoin = []
    for j in range(p['tiempos'][run]-1,-1,-1):
        filename = path_base + 'PNG/' + imagenData[2] + '/' + imagenData[2] + '_' + str(j) + '.png'        
        image_string = tf.io.read_file(filename)
        img_decoded = tf.io.decode_png(image_string, dtype=tf.uint16, channels=3)       
        
        if DA:
            img_decoded = applyDA(img_decoded, item['DA'])
                
        timeJoin.insert(0,img_decoded[int(imagenData[1]) - size:int(imagenData[1]) + size,
                                      int(imagenData[0]) - size:int(imagenData[0]) + size,
                                      0:p['canales'][run]])
 
        
    if p['tiempos'][run]==1:
        imagenData = tf.reshape(timeJoin[0],(p['margen'][run],p['margen'][run],p['canales'][run]))
    else:
        if p['meanMatrizImagen']:        
            img = tf.reduce_mean( timeJoin , axis=0 )
            imagenData = tf.reshape(img,(p['margen'][run],p['margen'][run],p['canales'][run]))
        else:
            img = tf.stack(timeJoin, axis=0)
            imagenData = tf.reshape(img,(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
        
    
    
    if len(p['inputs']) == 1:
        return imagenData, int(value)
    
    item['imagen'] = imagenData
    itemL = []
    for inpL in p['inputs']:
        itemL.append(item[inpL])
    
    return tuple(itemL), int(value)

In [105]:
def splitDataset(p, run, dataset, path_imagenes, products, times,val_split= 0.2):
   
    if p['dsVal']:
        train =  dataset
        test = pd.read_csv(p['dsVal'])
    else:
        # Escojemos una fraccion del dataset si se ha indicado   
        if p['dataset']:
            train, test = train_test_split(dataset.sample(frac=p['dataset']), test_size=val_split, shuffle=True)
        else:
            train, test = train_test_split(dataset, test_size=val_split, shuffle=True)
               
    inputsList = {}
    
    # Agregamos un atributo para indicar que el dato va realizar DA
    if p['DA']:        
        inputsList['DA'] = train['DA'].tolist() 
        
        
    print(f'Tamaño del dataset: Train {len(train)}  - Val {len(test)}')    
    
    for inp in p['inputs']:
        inputsList[inp] = train[inp].tolist()        
        
       
        
    train_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[p['outputs']].tolist()))           
    val_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[p['outputs']].tolist()))     
    
    train_dataset = train_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times,p['DA']))
    val_dataset = val_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times,p['DA']))
    
    train_dataset = train_dataset.batch(p['batch'])#.cache().prefetch(tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(p['batch'])#.prefetch(tf.data.AUTOTUNE)  
    
    return train_dataset, val_dataset

In [106]:
def crearDir(path, newDir):
    try:
        pathT = os.path.join(path, newDir)
        os.mkdir(pathT)
        return pathT
    except FileExistsError:
        return pathT
        pass
    except:
        print(f"No se pudo crear el directorio: {newDir}")
        pritn(f'Path base: {path}')
        pritn(f'Nuevo    : {newDir}')        
        return None

In [107]:
def inicializarVariables(repDir, params,ds):
    # Leemos dataset
    try:
        dataset = pd.read_csv(ds)        
        dsName = ds.split('/')[-1][:-4]
    except:
        print(f'No se pudo leer el dataset {ds}')
        return None
    
    statsDir = crearDir(repDir, dsName)
    excelFile = f'{statsDir}/Stats_{params["redTipo"]}_{params["outputs"]}_{datetime.today().strftime("%Y%m")}.xlsx'
    
    if params['record'] and not os.path.exists(excelFile):
        writer = pd.ExcelWriter(excelFile, engine = 'xlsxwriter')
        keys_values = params.items()
        strParams = {str(key): str(value) for key, value in keys_values}
        strParams['dsName'] = dsName
        pd.DataFrame(strParams,index=[0]).to_excel(writer, sheet_name = 'Informacion')          
        writer.save()
        
        
    return statsDir, excelFile, dataset            

In [108]:
def getCM(logs):
    lKeys = list(logs.keys())
    
    try:
        TN = int(logs[[x for x in lKeys if 'val_true_negatives' in x][0]])
        TP = int(logs[[x for x in lKeys if 'val_true_positives' in x][0]])
        FN = int(logs[[x for x in lKeys if 'val_false_negatives' in x][0]])
        FP = int(logs[[x for x in lKeys if 'val_false_positives' in x][0]])
    except:
        print(f'\nNo se pudo leer keys para la matriz de confucion en logs : {lKeys}')
        print(f'Se intento leer: val_true_negatives,val_true_positives, val_false_negatives y val_false_positives')
    
       
    y_true =  [0]*TN + [1]*TP + [1]*FN + [0]*FP
    _y_pred = [0]*TN + [1]*TP + [0]*FN + [1]*FP
    
    return TN, FP, FN, TP, np.array(y_true), np.array(_y_pred)

In [109]:
class CustomCB(Callback):
    """ Custom callback to compute metrics at the end of each training epoch"""
    def __init__(self, val_ds=None, WANDB=True):     
        self.val_ds = val_ds  
        self.history = {}
        self.wandb = WANDB
   
   
    def on_epoch_end(self, epoch, logs={}):       
        TN, FP, FN, TP, y_true, _y_pred = getCM(logs)
        
        self.history.setdefault('loss', []).append(logs['loss']) 
        self.history.setdefault('acc', []).append(logs['acc'])  
        self.history.setdefault('val_loss', []).append(logs['val_loss']) 
        self.history.setdefault('val_acc', []).append(logs['val_acc']) 
        
        
        self.history.setdefault('val_TN', []).append(TN) 
        self.history.setdefault('val_FP', []).append(FP)
        self.history.setdefault('val_FN', []).append(FN) 
        self.history.setdefault('val_TP', []).append(TP) 
        
            
        if self.wandb:
            wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                                    preds=_y_pred, y_true=y_true,
                                    class_names=[0,1]),                   
                       'val_TN' :TN,'val_FN' :FN,'val_TP' :TP,'val_FP' :FP,
                       'val_acc': logs['val_acc'],'loss' : logs['loss'],
                       'val_loss': logs['val_loss'],'acc' : logs['acc']                  
                      })


In [110]:
def crearCallbacks(statsDir,dsName, params,run, metricas):
    CB = metricas['callbacks']
    
    idModel = datetime.today().strftime("%Y%m%d_%H%M%S")
    checkpoint_path = f'{statsDir}/Model_{params["redTipo"]}_{params["outputs"]}_{idModel}.hdf5' 
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,                                                     
                                                     verbose=1)
    
    # Iniciamos WANDB
    if params['record']:        
        CB.append(cp_callback)   
     
    if params['WANDB']:
        config = dict(learning_rate=params['lr'], epochs = params['epocas'],
             batch_size =params['batch'],architecture="CNN", 
             num_classes = params['num_class'],)
        wandb.init(project=f'{params["Proyect"]}-({params["redTipo"]}-{params["outputs"]}-{len(params["inputs"])})',            
                   config=config,
                   name= f'Ex_{dsName}_({params["canales"][run]}-{params["tiempos"][run]}-{params["margen"][run]})_{idModel}')   
                                               
    return CB, idModel

In [128]:
def trainModel(params,path_imagenes, path_base ,products, times):    
         
    ds_files = [f'{params["dsDir"]}{e}' for e in os.listdir(params["dsDir"])]
    print(f'Cantidad de datasets : {len(ds_files)}')    
    print(f'Datasets a usar : {params["maxDS"]}')    
        
    
    all_result = []      
    
    repDir = crearDir(f'{path_base}/Archivos/Modelos', params["redTipo"])
    repDir = crearDir(repDir, params["Proyect"])
    
    print(f'DIRECTORIO BASE : {repDir}')
    
    # Una iteracion por cada dataset existente
    ds_i = 0
    for ds in ds_files[0:4]:#params['maxDS']]:  
        ds_i += 1       
        
        # Crear carpeta para DS y archivo de stats y el ds
        statsDir , statsFile , dataset = inicializarVariables(repDir, params,ds)         
            
        resultados = []    
        for run in range(params['runs']): 
            print('__________________________________________________')
            print(f'DATASET : {ds_i}/{params["maxDS"]}')            
            print('__________________________________________________')
            print(f'Inicio de la prueba N°: {(run+1)}/{params["runs"]}') 
            print(f'Dataset: {ds.split(".")[0]}')
            print(f'- Batch size:  {params["batch"]}')
            print('__________________________________________________')
            
           
            # Modelo 
            model = crearModelo(params,run) 
            metricas = getMetrics(params['redTipo'], params['lr'], params['paciencia'])
            model.compile(optimizer=metricas['optimizer'],loss=metricas['loss_fn'],metrics=metricas['metrics'],)
            
            # Dataset        
            train_dataset, val_dataset = splitDataset(params,run, dataset, path_imagenes, products, times, params['val_split'])
          
            
            # Creamos los callbacks
            CB, idModel = crearCallbacks(statsDir,ds.split('/')[-1][:-4], params,run, metricas)           

            if params['redTipo'] == 'clasificacion':
                hist =  CustomCB(val_dataset, params['WANDB'])
                CB.append(hist)
            
            
            #Entrenamos
            history = model.fit(train_dataset,batch_size=params['batch'],                            
                                epochs=params['epocas'],callbacks=CB,
                                validation_data=val_dataset,
                                validation_batch_size=params['batch'],
                                verbose=1)
            if params['redTipo'] != 'clasificacion': 
                HistTemp = history
            else:
                HistTemp = hist
                
            resultados.append(HistTemp.history)
            
            # Guardamos las estadisticas
            if params['record']:                
                with pd.ExcelWriter(statsFile, mode="a", engine="openpyxl", if_sheet_exists='overlay') as writer:                    
                    pd.DataFrame(HistTemp.history).to_excel(writer,startrow=0,
                                                           sheet_name=f'{run}-({params["canales"][run]}-{params["tiempos"][run]}-{params["margen"][run]})-{idModel}')
            if params['WANDB']:
                wandb.finish()
        
        all_result.append(resultados)
        
    return all_result

In [129]:
import wandb
from wandb.keras import WandbCallback
wandb.login()


True

In [130]:
"""
# Definimos las varibles para las iteraciones
Los parametros que van a cambiar son:
- Canales (products)
- Tiempos (Min de las imagenes)
- margen
"""

products = ['C07','C08','C13']
times   = ['10','20','30','40','50','00']


p_train = {
            # Reportes
          'Proyect'  : 'TesisDiego3D_DA',#'TesisDiego3D_V3', # TesisDiego
          'record'   : True,  # Grabar los resultados en  excels    
          'WANDB'    : False, # Grabar los resultados en WANDB
    
            # Datos del modelo
          'redTipo'  : 'clasificacion', # clasificacion / regresion
          'inputs'   : ['imagen', 'dato'], #altura]
          'outputs'  : 'clase',
          'num_class': 2,             # Solo se usa en el modelo de clasificacion
          'meanMatrizImagen' : False, # True -> Usa una conv2d, caso contrario conv3d                     
          
        
            # Variables del entrenamiento          
          'lr'       : 0.001,
          'batch'    : 64,        
          'val_split': 0.2,     # ¡¡ Si dsVal existe, este valor se ignora !!
          'epocas'   : 15,  
          'paciencia': 7,        # 0 = No paciencia  
    
           # Dataset
          'dsDir'    : f'{path_base}/Archivos/Dataset/ClasificacionDA/',
          'dsVal'    : f'{path_base}/Archivos/Dataset/Clase_DA_ValidDS.csv',
          'maxDS'    : -1,      #-1 = Todos
          'dataset'  : 1,     # 1 = 100% del ds
          'DA'       : True,     
                    
           # Hiper parametros 
          'canales'  : [3,2],
          'tiempos'  : [6,6],
          'margen'   : [30,30],
          'runs'     : 2
         }

In [131]:
# Forma del DS
ds_files = [f'{p_train["dsDir"]}{e}' for e in os.listdir(p_train["dsDir"])]
if p_train['dsVal']:
    dfVal = pd.read_csv(p_train['dsVal'])
    print(f'VALDIACION : {len(dfVal)}')
    print('--------------------------------------------------')
print(f'Cantidad de datasets : {len(ds_files)}')  
tempDF = pd.read_csv(ds_files[0])
print(f'TRAIN: {len(tempDF)}')
tempDF.head(2)

VALDIACION : 3435
--------------------------------------------------
Cantidad de datasets : 9
TRAIN: 6856


,Unnamed: 0,nombre,codigo,XO,XA,longitud,latitud,altura,dato,90%,99%,75%,fecha,flag,flagV2,imagen,clase,DA
0,0,EL TAMBO,47E0A2A8,269,497,-78.70694,-7.57417,700.0,0.0,0.0,0.2,0.0,2021-12-20-11,C0000002,C01,269--497--2021-12-20-11,1,0
1,1,EL TAMBO,47E0A2A8,269,497,-78.70694,-7.57417,700.0,0.0,0.0,0.2,0.0,2021-12-10-12,C0000002,C01,269--497--2021-12-10-12,1,0


In [ ]:
%%time
resultados = trainModel(p_train,path_imagenes,path_base,products,times)

Cantidad de datasets : 9
Datasets a usar : -1
DIRECTORIO BASE : C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA
__________________________________________________
DATASET : 1/-1
__________________________________________________
Inicio de la prueba N°: 1/2
Dataset: C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Dataset/ClasificacionDA/CLASE_TrainDS_0
- Batch size:  64
__________________________________________________
Creando modelo 3D
HP :(T:6 - M:30 - C:3) y  tipo (clasificacion)
Tamaño del dataset: Train 6856  - Val 3435
Epoch 1/15
107/108 [============================>.] - ETA: 2s - loss: 16.5383 - acc: 0.9338 - true_positives_23: 3387.0000 - true_negatives_23: 3008.0000 - false_positives_23: 412.0000 - false_negatives_23: 41.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA\CLASE_TrainDS_0\Model_clasificacion_clase_20220430_145000.hdf5
108/10

107/108 [============================>.] - ETA: 2s - loss: 3.7906 - acc: 0.7811 - true_positives_24: 2596.0000 - true_negatives_24: 2753.0000 - false_positives_24: 667.0000 - false_negatives_24: 832.0000
Epoch 2: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA\CLASE_TrainDS_0\Model_clasificacion_clase_20220430_160355.hdf5
108/108 [==============================] - 625s 6s/step - loss: 3.7869 - acc: 0.7814 - true_positives_24: 2596.0000 - true_negatives_24: 2761.0000 - false_positives_24: 667.0000 - false_negatives_24: 832.0000 - val_loss: 0.6976 - val_acc: 0.5000 - val_true_positives_24: 0.0000e+00 - val_true_negatives_24: 3428.0000 - val_false_positives_24: 0.0000e+00 - val_false_negatives_24: 3428.0000
Epoch 3/15
107/108 [============================>.] - ETA: 3s - loss: 0.7110 - acc: 0.4994 - true_positives_24: 0.0000e+00 - true_negatives_24: 3420.0000 - false_positives_24: 0.0000e+00 - false_negatives_24: 3428.0000
E

Epoch 4/15
107/108 [============================>.] - ETA: 2s - loss: 0.6842 - acc: 0.3462 - true_positives_25: 420.0000 - true_negatives_25: 1951.0000 - false_positives_25: 1469.0000 - false_negatives_25: 3008.0000
Epoch 4: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA\CLASE_TrainDS_1\Model_clasificacion_clase_20220430_172535.hdf5
108/108 [==============================] - 512s 5s/step - loss: 0.6840 - acc: 0.3470 - true_positives_25: 420.0000 - true_negatives_25: 1959.0000 - false_positives_25: 1469.0000 - false_negatives_25: 3008.0000 - val_loss: 0.6784 - val_acc: 0.5000 - val_true_positives_25: 0.0000e+00 - val_true_negatives_25: 3428.0000 - val_false_positives_25: 0.0000e+00 - val_false_negatives_25: 3428.0000
Epoch 5/15
107/108 [============================>.] - ETA: 3s - loss: 0.6818 - acc: 0.3592 - true_positives_25: 567.0000 - true_negatives_25: 1893.0000 - false_positives_25: 1527.0000 - false_negatives_25: 2

Epoch 6/15
107/108 [============================>.] - ETA: 3s - loss: 0.6535 - acc: 0.6542 - true_positives_26: 3428.0000 - true_negatives_26: 1052.0000 - false_positives_26: 2368.0000 - false_negatives_26: 0.0000e+00
Epoch 6: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA\CLASE_TrainDS_1\Model_clasificacion_clase_20220430_185733.hdf5
108/108 [==============================] - 814s 8s/step - loss: 0.6530 - acc: 0.6546 - true_positives_26: 3428.0000 - true_negatives_26: 1060.0000 - false_positives_26: 2368.0000 - false_negatives_26: 0.0000e+00 - val_loss: 1.0219 - val_acc: 0.5000 - val_true_positives_26: 0.0000e+00 - val_true_negatives_26: 3428.0000 - val_false_positives_26: 0.0000e+00 - val_false_negatives_26: 3428.0000
Epoch 7/15
107/108 [============================>.] - ETA: 3s - loss: 0.8103 - acc: 0.2975 - true_positives_26: 1892.0000 - true_negatives_26: 145.0000 - false_positives_26: 3275.0000 - false_negatives_2

Epoch 1/15
107/108 [============================>.] - ETA: 3s - loss: 18.1402 - acc: 0.9230 - true_positives_27: 3390.0000 - true_negatives_27: 2931.0000 - false_positives_27: 489.0000 - false_negatives_27: 38.0000
Epoch 1: saving model to C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis/Archivos/Modelos\clasificacion\TesisDiego3D_DA\CLASE_TrainDS_2\Model_clasificacion_clase_20220430_215411.hdf5
108/108 [==============================] - 667s 6s/step - loss: 18.1191 - acc: 0.9231 - true_positives_27: 3390.0000 - true_negatives_27: 2939.0000 - false_positives_27: 489.0000 - false_negatives_27: 38.0000 - val_loss: 142.4989 - val_acc: 0.5000 - val_true_positives_27: 0.0000e+00 - val_true_negatives_27: 3428.0000 - val_false_positives_27: 0.0000e+00 - val_false_negatives_27: 3428.0000
Epoch 2/15
107/108 [============================>.] - ETA: 2s - loss: 12.0723 - acc: 0.7220 - true_positives_27: 2276.0000 - true_negatives_27: 2668.0000 - false_positives_27: 752.0000 - false_negatives_27: 